In [24]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [25]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

# תרגיל: דוח תפעול יומי למערך גלאים

בתרגיל הזה נתרגל עבודה עם **Pandas** דרך סימולציה של מצב אמיתי מעולמות הפיזיקה הניסויית:  
מערך גלאים (למשל קרינה/קרניים קוסמיות) שמייצר **לוג אירועים** לאורך זמן, ובמקביל מערכת ניטור סביבתי שמודדת **טמפרטורה** מכמה פרובים.

המטרה שלנו היא לקחת נתונים “גולמיים” (Raw Logs) — שכוללים לפעמים **טיימסטמפים לא תקינים**, **ערכים חסרים**, וסדר לא מסודר — ולהפיק מהם **דוח יומי** נקי ושמיש.

במהלך התרגיל תתרגלו:

- **ניקוי נתונים (Data Cleaning)**  
  המרת עמודת זמן למבנה datetime עם `pd.to_datetime(..., errors="coerce")`, סילוק שורות בעייתיות, וטיפול בערכים חסרים.

- **עבודה עם תאריכים וזמן (Date/Time)**  
  חילוץ תאריך יומי מתוך timestamp (לצורך סיכום לפי ימים).

- **Pivot Tables**  
  יצירת טבלת דוח רחבה של *טמפרטורה ממוצעת לפי יום ולפי פרוב* בעזרת `pivot_table`.

- **GroupBy + size + unstack**  
  ספירת מספר אירועים לפי יום ולפי גלאי, והפיכת התוצאה לטבלה רחבה שמתאימה לדוח.

- **Merge / Join (מתקדם)**  
  חיבור לוג האירועים לטבלת מטא-דאטה של גלאים (קליברציה), כדי להמיר מדידה גולמית לערך פיזיקלי “מחושב”.

- **טרנספורמציות ועמודות נגזרות (Vectorized transforms)**  
  חישוב אנרגיה מכוילת לכל אירוע בצורה וקטורית (ללא לולאות).

- **שינויי מבנה (Reshaping)**  
  מעבר בין פורמט רחב לפורמט ארוך (`melt`) לצורך גרפים/ניתוחים.

- **ויזואליזציה מהירה**  
  גרפים בסיסיים של ספירות אירועים וטמפרטורות לאורך זמן.

התרגיל מדמה Workflow נפוץ במעבדה:
1. קבלת לוגים ממכשור מדידה (אירועים + סביבה)
2. ניקוי ותיקון נתונים
3. סיכום יומי לפי חיישנים/גלאים
4. שילוב נתוני קליברציה/מטא-דאטה
5. הפקת דוח תפעולי שמאפשר לזהות תקלות, חריגות ועקביות לאורך זמן

בסוף התרגיל יהיו לכם כמה “טבלאות דוח” מרכזיות + גרפים קצרים — בדיוק כמו בדוח ניטור יומי של מערכת מדידה אמיתית.

## חלק 0 – יצירת נתוני ניסוי

קטע הקוד יוצר **סט נתונים סינתטי** המדמה לוגים של ניסוי פיזיקלי אמיתי, עם דגש על ריאליזם ולא על “נתונים נקיים”.

נטענות ספריות העבודה הסטנדרטיות, ומאותחל מחולל מספרים אקראיים עם seed קבוע לצורך שחזור תוצאות.

נבנית טבלת מטא-דאטה של גלאים, הכוללת שיוך למודולים ופרמטרי קליברציה (gain, offset), כפי שמקובל במערכות מדידה אמיתיות.

לאחר מכן נוצר לוג אירועים:
אירועים מתרחשים בזמנים אקראיים ולא אחידים לאורך כמה ימים, לכל אירוע משויכים גלאי וערך מדידה גולמי (ADC) עם רעש.

בכוונה מוכנסים לנתונים:
- חותמות זמן לא תקינות  
- ערכי מדידה חסרים  

בנוסף נוצר לוג נפרד של מדידות טמפרטורה מכמה פרובים סביבתיים, גם הוא כולל רעש, ערכים חסרים וחותמות זמן שגויות.

לבסוף מודפסים גדלי הטבלאות כדי לקבל תמונת מצב ראשונית על היקף הנתונים לפני שלב הניקוי והעיבוד.

הקטע כולו מדמה נקודת התחלה של ניתוח נתונים ניסויי, שעליה נבצע בהמשך ניקוי, סיכום וניתוח באמצעות Pandas.



In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -----------------------------
# 0) Generate synthetic-but-realistic data (raw, slightly messy)
# -----------------------------
rng = np.random.default_rng(42)

# Detector metadata / calibration
detectors = pd.DataFrame({
    "detector": [f"D{i}" for i in range(1, 7)],
    "module":   ["M1", "M1", "M2", "M2", "M3", "M3"],
    "gain":     rng.uniform(0.8, 1.4, size=6),
    "offset":   rng.uniform(-5.0, 5.0, size=6),
})

# Create a time range spanning multiple days, but events are irregular
start = pd.Timestamp("2025-12-20 00:00:00")
end   = pd.Timestamp("2025-12-26 23:59:59")
total_seconds = int((end - start).total_seconds())

n_events = 3500
event_times = start + pd.to_timedelta(rng.integers(0, total_seconds, size=n_events), unit="s")
event_times = pd.Series(event_times).sample(frac=1, random_state=0).reset_index(drop=True)  # shuffled

events = pd.DataFrame({
    "timestamp": event_times.astype(str),  # stored as strings in raw logs
    "detector": rng.choice(detectors["detector"], size=n_events, replace=True),
    "adc": rng.gamma(shape=4.0, scale=35.0, size=n_events) + rng.normal(0, 3, size=n_events),
})

# Add some deliberately invalid timestamps and a couple junk rows
bad_idx = rng.choice(events.index, size=25, replace=False)
events.loc[bad_idx, "timestamp"] = rng.choice(
    ["not_a_time", "2025-13-99 99:99:99", "", "2025/12/25 25:61:00"],
    size=len(bad_idx),
)
# Also introduce some missing adc values
events.loc[rng.choice(events.index, size=15, replace=False), "adc"] = np.nan

# Temperature probe logs (more regular, but still messy)
probes = ["T1", "T2", "T3"]
n_temps = 1400
temp_times = start + pd.to_timedelta(rng.integers(0, total_seconds, size=n_temps), unit="s")

temps = pd.DataFrame({
    "timestamp": pd.Series(temp_times).astype(str),
    "probe": rng.choice(probes, size=n_temps, replace=True),
    "temp_C": rng.normal(loc=24.0, scale=1.7, size=n_temps),
})

# Add a few invalid timestamps and missing temps
bad_idx_t = rng.choice(temps.index, size=12, replace=False)
temps.loc[bad_idx_t, "timestamp"] = rng.choice(["bad", "2025-12-XX 12:00:00", ""], size=len(bad_idx_t))
temps.loc[rng.choice(temps.index, size=10, replace=False), "temp_C"] = np.nan

print("Raw shapes:")
print("  detectors:", detectors.shape)
print("  events:", events.shape)
print("  temps:", temps.shape)

Raw shapes:
  detectors: (6, 4)
  events: (3500, 3)
  temps: (1400, 3)


## חלק 1 – ניקוי נתוני האירועים והטמפרטורות

בקטע הזה אנחנו יוצרים גרסאות “נקיות” של שתי הטבלאות (`events`, `temps`) כדי שיהיה אפשר לעבוד איתן בצורה אמינה:

- קודם כל עושים `copy()` כדי לא לשנות את הנתונים הגולמיים.
- ממירים את עמודת הזמן למבנה datetime בעזרת `pd.to_datetime`, כאשר `errors="coerce"` הופך ערכים לא תקינים ל־`NaT` (כלומר “זמן חסר”).
- מסירים שורות בעייתיות באמצעות `dropna`:  
  בלוג האירועים מסירים שורות עם timestamp לא תקין או ערך `adc` חסר,  
  ובלוג הטמפרטורות מסירים שורות עם timestamp לא תקין או `temp_C` חסר.
- לבסוף מוסיפים עמודת `date` שמכילה רק את התאריך (בלי שעה), כדי לאפשר סיכומים יומיים (daily rollup) בהמשך.


In [27]:
# -----------------------------
# 1) Clean + parse timestamps (core exam skill: pd.to_datetime, errors='coerce')
# -----------------------------
# TODO: Create a clean copy of the raw events table (do not modify the original)
# events_clean = ...
# TODO: Convert the timestamp column to datetime; invalid strings should become NaT
# events_clean["timestamp"] = ...
# TODO: Drop rows with invalid timestamps (NaT)
# events_clean = ...
# TODO: Drop rows with missing ADC measurements
# events_clean = ...
# TODO: Extract the calendar date for daily aggregations
# events_clean["date"] = ...


# TODO: Create a clean copy of the raw temperature table (do not modify the original)
# temps_clean = ...
# TODO: Convert the timestamp column to datetime; invalid strings should become NaT
# temps_clean["timestamp"] = ...
# TODO: Drop rows with invalid timestamps (NaT)
# temps_clean = ...
# TODO: Drop rows with missing temperature measurements
# temps_clean = ...
# TODO: Extract the calendar date for daily aggregations
# temps_clean["date"] = ...

try:
    print("\nAfter cleaning:")
    print("  events_clean:", events_clean.shape)
    print("  temps_clean:", temps_clean.shape)
except:
    pass


After cleaning:


## חלק 2 – טבלת טמפרטורה יומית לכל פרוב (Pivot Table)

כאן אנחנו מסכמים את נתוני הטמפרטורה הנקיים לטבלת דוח “רחבה”:
- כל שורה מייצגת **תאריך** (`date`)
- כל עמודה מייצגת **פרוב טמפרטורה** (`probe`)
- בתאים מופיעה **הטמפרטורה הממוצעת** לאותו יום ולאותו פרוב (`mean` על `temp_C`)

הפעולה נעשית עם `pivot_table`, ואז אנחנו ממיינים את האינדקס (ימים) ואת העמודות (הפרובים) כדי לקבל טבלה מסודרת לקריאה. לבסוף מדפיסים דוגמה קצרה עם `head()`.


In [28]:
# -----------------------------
# 2) Daily mean temperature per probe (exam skill: pivot_table)
# -----------------------------

# TODO: Create a pivot table (wide format) that summarizes the cleaned temperature logs.
#       Input table: temps_clean with columns ["timestamp", "probe", "temp_C", "date"].
#       We want a DAILY summary, so:
#         - index should be "date"  -> each row is one calendar day
#         - columns should be "probe" -> each column is one temperature sensor/probe (T1/T2/T3...)
#         - values should be "temp_C" -> the numeric measurement we aggregate
#         - aggfunc should be "mean"  -> average temperature per (day, probe)
#       Result: a report-like table where you can quickly compare probes across days.
# daily_mean_temp = ...

# TODO: Sort the resulting report table for consistent, readable output.
#       - sort_index() ensures dates are in chronological order (oldest -> newest).
#       - sort_index(axis=1) ensures probe columns are in a consistent order (e.g., T1, T2, T3).
#       Tip: You can either chain both sorts after pivot_table, or do them in two steps.
# daily_mean_temp = ...

try:
    print("\nDaily mean temperature (date x probe):")
    print(daily_mean_temp.head())
except:
    pass


Daily mean temperature (date x probe):


## חלק 3 – ספירת אירועים יומית לכל גלאי (groupby + size + unstack)

כאן אנחנו בונים טבלת דוח שמראה **כמה אירועים נמדדו בכל יום בכל גלאי**:

- מבצעים `groupby` לפי שני מפתחות: `date` (יום) ו־`detector` (גלאי), כדי לחלק את האירועים לקבוצות לפי (יום, גלאי).
- משתמשים ב־`size()` כדי לקבל **ספירה** של מספר השורות בכל קבוצה (כלומר מספר האירועים).
- הופכים את התוצאה לטבלה רחבה בעזרת `unstack`:  
  השורות יהיו תאריכים, העמודות יהיו גלאים, והתאים יהיו מספר האירועים.
- `fill_value=0` מבטיח שאם ביום מסוים לא היו אירועים בגלאי מסוים, נקבל 0 ולא ערך חסר.
- בסוף ממיינים את האינדקס (ימים) והעמודות (גלאים) לקבלת דוח מסודר, ומדפיסים `head()` לבדיקת תקינות.


In [29]:
# -----------------------------
# 3) Daily event counts per detector (exam skill: groupby + size + unstack)
# -----------------------------
# TODO: Create a DAILY event-count report from events_clean in a single pipeline.
#       Use groupby(["date","detector"]) to group events by day and detector, then .size() to count rows per group.
#       Reshape the result into a wide table with .unstack(fill_value=0) so rows are dates, columns are detectors,
#       and missing (date, detector) combinations become 0. Finally, sort rows by date with sort_index() and
#       sort columns (detectors) with sort_index(axis=1). Print a short preview using head() with a clear label.
daily_counts = (
    
)

try:
    print("\nDaily event counts (date x detector):")
    print(daily_counts.head())
except:
    pass


Daily event counts (date x detector):


## חלק 4 – חיבור מטא-דאטה וחישוב אנרגיה מכוילת (merge + עמודה נגזרת)

בשלב הזה אנחנו עוברים מלוג גולמי לנתונים בעלי משמעות פיזיקלית יותר:

- מבצעים **merge** בין טבלת האירועים הנקייה (`events_clean`) לבין טבלת המטא-דאטה של הגלאים (`detectors`), כך שלכל אירוע נצמיד את פרטי הקליברציה של הגלאי שמדד אותו (module, gain, offset).
- לאחר החיבור מחשבים עמודה חדשה `energy` בצורה **וקטורית** (ללא לולאות):  
  \( E = gain \cdot adc + offset \)  
  זה מדמה המרה ממדידה גולמית (ADC) ל“אנרגיה” מכוילת.
- מוסיפים בדיקות בסיסיות (`assert`) כדי לוודא:
  1) שלא נשארו אירועים עם גלאי שלא נמצא בטבלת המטא-דאטה  
  2) שהאנרגיה שחושבה היא מספר סופי (לא NaN/inf)
- לבסוף מדפיסים תצוגה מקדימה של העמודות החשובות כדי לוודא שהחיבור והחישוב הצליחו.


In [30]:
# -----------------------------
# 4) Merge metadata + compute calibrated energy (Week 11-ish: merge + vectorized transform)
# -----------------------------
# TODO: Merge the cleaned events table with the detector metadata table.
#       Goal: for every row in events_clean, attach its detector's module/gain/offset from detectors.
#       Use a LEFT join so we keep all events even if metadata is missing (we'll detect that later).
#       Merge key: "detector" (exists in both tables).
# events_cal = ...

# TODO: Compute a calibrated "energy" column using a vectorized formula (no loops):
#       energy = gain * adc + offset
#       This simulates converting a raw ADC measurement into a calibrated physical quantity.
# events_cal["energy"] = ...

# Basic sanity checks
try:
    assert events_cal["module"].notna().all(), "Some detectors did not match metadata!"
    assert np.isfinite(events_cal["energy"]).all(), "Energy contains non-finite values!"

    print("\nEvents with calibration columns:")
    print(events_cal[["timestamp", "date", "detector", "module", "adc", "gain", "offset", "energy"]].head())
except:
    pass

## חלק 5 – סיכום יומי לפי מודול (Multi-Aggregation עם groupby)

כאן אנחנו מסכמים את הנתונים המכיילים לרמת **מודולים** (ולא גלאים בודדים), כדי לקבל תמונת מצב תפעולית “גבוהה” יותר:

- מקבצים (`groupby`) לפי `date` ו־`module`, כך שכל קבוצה מייצגת מודול מסוים ביום מסוים.
- מתמקדים בעמודת `energy` ומפעילים עליה כמה אגרגציות במקביל בעזרת `agg`:
  - `n_events`: כמה אירועים נמדדו (ספירה)
  - `mean_energy`: אנרגיה ממוצעת
  - `max_energy`: האנרגיה המקסימלית שנמדדה באותו יום במודול
- לאחר מכן עושים `reset_index()` כדי להפוך את האינדקס לעמודות רגילות (נוח להמשך עבודה/הדפסה).
- ממיינים לפי תאריך ואז מודול כדי לקבל טבלה קריאה ומסודרת.
- לבסוף מציגים `head()` כדי לוודא שהמבנה והתוצאות הגיוניים.

התוצאה היא טבלה בפורמט “ארוך” (long format), שמתאימה גם לניתוח נוסף וגם ליצירת טבלאות/גרפים.


In [31]:
# -----------------------------
# 5) Daily module-level summary table (multi-agg)
# -----------------------------
# TODO: Build a daily, module-level summary table from the calibrated events data:
#       use the date and module columns to define groups, then summarize the calibrated measurements
#       within each group using multiple statistics (so the output has several informative columns).
#       After aggregating, convert the grouped result into a regular DataFrame, sort it in a consistent
#       report-friendly order, and print a small preview to confirm the structure and values look right.
module_daily_summary = (

)

try:
    print("\nModule daily summary (long format):")
    print(module_daily_summary.head())
except:
    pass


Module daily summary (long format):


## מעבר מפורמט “ארוך” לדוח “רחב” לפי מודולים

לאחר שיש לנו טבלת סיכום יומית בפורמט ארוך (`module_daily_summary`), אנחנו ממירים אותה לטבלת דוח רחבה ונוחה לקריאה:
- השורות הן תאריכים
- העמודות הן מודולים
- בכל תא מופיע הערך הממוצע היומי (למשל mean energy) עבור אותו מודול

ההמרה נעשית עם `pivot_table`, ואז ממיינים את התאריכים ואת שמות המודולים כדי לקבל פלט עקבי ומסודר. לבסוף מדפיסים `head()` כדי לוודא שהמבנה תקין.

In [32]:
# TODO: Create a wide, report-style table from module_daily_summary:
#       reshape the data so each row is a date and each column is a module, with cells showing the
#       (daily) average calibrated measurement for that module. Use a pivot-style operation to go
#       from the long format summary to a wide format report, then sort the date index chronologically
#       and sort the module columns consistently. Finally, print a short preview (head) to confirm
#       the output layout is "date x module" and the values look reasonable.

# Also make a wide "report" view: date x module (mean energy)
# module_mean_energy_wide = module_daily_summary.pivot_table(
# 
# )

try:
    print("\nMean energy per module (wide):")
    print(module_mean_energy_wide.head())
except:
    pass


Mean energy per module (wide):


## חלק 6 – מעבר לפורמט “ארוך” לצורך גרפים (melt)

כאן אנחנו ממירים טבלאות דוח “רחבות” (wide) לפורמט “ארוך” (long) שמתאים יותר לשרטוטים ולעבודה עם ספריות גרפים:

- `daily_counts` היה בפורמט שבו כל עמודה היא גלאי. בעזרת `reset_index()` ו־`melt` הופכים אותו לטבלה עם שלוש עמודות מרכזיות: תאריך, גלאי, וספירת אירועים.
- `daily_mean_temp` היה בפורמט שבו כל עמודה היא פרוב. באותו אופן, `melt` יוצר טבלה עם תאריך, פרוב, וטמפרטורה ממוצעת.

בפורמט הארוך כל שורה מייצגת תצפית אחת (תאריך + חיישן), מה שמקל לייצר גרפים כמו קווים/נקודות לפי קטגוריה.


In [33]:
# TODO: Convert the daily_counts wide report into a long/tidy table for plotting.
#       Goal: one row per (date, detector) with a single count value column.
#       Steps: move "date" out of the index (reset_index), then melt detector columns into two columns:
#       - a categorical column for detector IDs
#       - a numeric column for the event counts
counts_long = ...

# TODO: Convert the daily_mean_temp wide report into a long/tidy table for plotting.
#       Goal: one row per (date, probe) with a single mean temperature value column.
#       Steps: reset_index to expose "date", then melt probe columns into:
#       - a categorical column for probe IDs
#       - a numeric column for mean temperature values
temps_long = ...


## סיכום – הכנת הנתונים להצגה וסקירה ויזואלית של התוצאות

בחלק המסכם הזה אנחנו עוברים משלב עיבוד הנתונים לשלב **ההצגה והבדיקה**:

ראשית, טבלאות הדוח הרחבות מומרות לפורמט “ארוך” (`melt`). זהו פורמט סטנדרטי ונוח לויזואליזציה ולניתוחים המשכיים, שבו כל שורה מייצגת תצפית אחת (תאריך + חיישן/גלאי/מודול).

לאחר מכן מצוירים גרפים פשוטים באמצעות Pandas ו־Matplotlib, שמאפשרים לקבל במהירות תמונת מצב:
- כיצד מספר האירועים משתנה מיום ליום בכל גלאי
- כיצד הטמפרטורה הממוצעת משתנה בפרובים השונים
- כיצד האנרגיה הממוצעת (המכוילת) משתנה בין מודולים לאורך הזמן

לבסוף, כל תוצרי הביניים והדוחות הסופיים נאספים למילון אחד (`report`). זה מדמה שלב סיום של ניתוח ניסויי: ריכוז כל הטבלאות הרלוונטיות במקום אחד לצורך בדיקות, המשך עיבוד או הגשה כדוח.

קטע זה משלים את ה־Workflow המלא: מנתונים גולמיים ורועשים → ניקוי ועיבוד → סיכום יומי → הצגה ויזואלית ותוצר סופי.


In [34]:
try:
    # Plot A: Daily counts by detector
    ax = daily_counts.plot(kind="line", marker="o", figsize=(10, 4))
    ax.set_title("Daily event counts per detector")
    ax.set_xlabel("Date")
    ax.set_ylabel("Count")
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Plot B: Daily mean temperature per probe
    ax = daily_mean_temp.plot(kind="line", marker="o", figsize=(10, 4))
    ax.set_title("Daily mean temperature per probe")
    ax.set_xlabel("Date")
    ax.set_ylabel("Temperature (°C)")
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Plot C: Mean calibrated energy per module
    ax = module_mean_energy_wide.plot(kind="line", marker="o", figsize=(10, 4))
    ax.set_title("Daily mean calibrated energy per module")
    ax.set_xlabel("Date")
    ax.set_ylabel("Mean energy (arb. units)")
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    report = {
        "daily_mean_temp": daily_mean_temp,
        "daily_counts": daily_counts,
        "module_daily_summary_long": module_daily_summary,
        "module_mean_energy_wide": module_mean_energy_wide,
        "counts_long": counts_long,
        "temps_long": temps_long,
    }

    print("\nReport keys:", list(report.keys()))
except:
    pass

`````{admonition} Solutions
:class: dropdown, tip
```python
# Physics-inspired Pandas notebook (FULL SOLUTION CODE)
# Problem 1: "Detector Array Daily Operations Report"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -----------------------------
# 0) Generate synthetic-but-realistic data (raw, slightly messy)
# -----------------------------
rng = np.random.default_rng(42)

# Detector metadata / calibration
detectors = pd.DataFrame({
    "detector": [f"D{i}" for i in range(1, 7)],
    "module":   ["M1", "M1", "M2", "M2", "M3", "M3"],
    "gain":     rng.uniform(0.8, 1.4, size=6),
    "offset":   rng.uniform(-5.0, 5.0, size=6),
})

# Create a time range spanning multiple days, but events are irregular
start = pd.Timestamp("2025-12-20 00:00:00")
end   = pd.Timestamp("2025-12-26 23:59:59")
total_seconds = int((end - start).total_seconds())

n_events = 3500
event_times = start + pd.to_timedelta(rng.integers(0, total_seconds, size=n_events), unit="s")
event_times = pd.Series(event_times).sample(frac=1, random_state=0).reset_index(drop=True)  # shuffled

events = pd.DataFrame({
    "timestamp": event_times.astype(str),  # stored as strings in raw logs
    "detector": rng.choice(detectors["detector"], size=n_events, replace=True),
    "adc": rng.gamma(shape=4.0, scale=35.0, size=n_events) + rng.normal(0, 3, size=n_events),
})

# Add some deliberately invalid timestamps and a couple junk rows
bad_idx = rng.choice(events.index, size=25, replace=False)
events.loc[bad_idx, "timestamp"] = rng.choice(
    ["not_a_time", "2025-13-99 99:99:99", "", "2025/12/25 25:61:00"],
    size=len(bad_idx),
)
# Also introduce some missing adc values
events.loc[rng.choice(events.index, size=15, replace=False), "adc"] = np.nan

# Temperature probe logs (more regular, but still messy)
probes = ["T1", "T2", "T3"]
n_temps = 1400
temp_times = start + pd.to_timedelta(rng.integers(0, total_seconds, size=n_temps), unit="s")

temps = pd.DataFrame({
    "timestamp": pd.Series(temp_times).astype(str),
    "probe": rng.choice(probes, size=n_temps, replace=True),
    "temp_C": rng.normal(loc=24.0, scale=1.7, size=n_temps),
})

# Add a few invalid timestamps and missing temps
bad_idx_t = rng.choice(temps.index, size=12, replace=False)
temps.loc[bad_idx_t, "timestamp"] = rng.choice(["bad", "2025-12-XX 12:00:00", ""], size=len(bad_idx_t))
temps.loc[rng.choice(temps.index, size=10, replace=False), "temp_C"] = np.nan

print("Raw shapes:")
print("  detectors:", detectors.shape)
print("  events:", events.shape)
print("  temps:", temps.shape)

# -----------------------------
# 1) Clean + parse timestamps (core exam skill: pd.to_datetime, errors='coerce')
# -----------------------------
events_clean = events.copy()
events_clean["timestamp"] = pd.to_datetime(events_clean["timestamp"], errors="coerce")
events_clean = events_clean.dropna(subset=["timestamp"])  # remove invalid timestamps
events_clean = events_clean.dropna(subset=["adc"])        # remove missing adc
events_clean["date"] = events_clean["timestamp"].dt.date  # daily rollup key

temps_clean = temps.copy()
temps_clean["timestamp"] = pd.to_datetime(temps_clean["timestamp"], errors="coerce")
temps_clean = temps_clean.dropna(subset=["timestamp"])
temps_clean = temps_clean.dropna(subset=["temp_C"])
temps_clean["date"] = temps_clean["timestamp"].dt.date

print("\nAfter cleaning:")
print("  events_clean:", events_clean.shape)
print("  temps_clean:", temps_clean.shape)

# -----------------------------
# 2) Daily mean temperature per probe (exam skill: pivot_table)
# -----------------------------
daily_mean_temp = temps_clean.pivot_table(
    index="date",
    columns="probe",
    values="temp_C",
    aggfunc="mean"
).sort_index().sort_index(axis=1)

print("\nDaily mean temperature (date x probe):")
print(daily_mean_temp.head())

# -----------------------------
# 3) Daily event counts per detector (exam skill: groupby + size + unstack)
# -----------------------------
daily_counts = (
    events_clean
    .groupby(["date", "detector"])
    .size()
    .unstack(fill_value=0)
    .sort_index()
    .sort_index(axis=1)
)

print("\nDaily event counts (date x detector):")
print(daily_counts.head())

# -----------------------------
# 4) Merge metadata + compute calibrated energy (Week 11-ish: merge + vectorized transform)
# -----------------------------
events_cal = events_clean.merge(detectors, on="detector", how="left")

# Calibrated energy proxy: E = gain*adc + offset
events_cal["energy"] = events_cal["gain"] * events_cal["adc"] + events_cal["offset"]

# Basic sanity checks
assert events_cal["module"].notna().all(), "Some detectors did not match metadata!"
assert np.isfinite(events_cal["energy"]).all(), "Energy contains non-finite values!"

print("\nEvents with calibration columns:")
print(events_cal[["timestamp", "date", "detector", "module", "adc", "gain", "offset", "energy"]].head())

# -----------------------------
# 5) Daily module-level summary table (multi-agg)
# -----------------------------
module_daily_summary = (
    events_cal
    .groupby(["date", "module"])["energy"]
    .agg(
        n_events="size",
        mean_energy="mean",
        max_energy="max"
    )
    .reset_index()
    .sort_values(["date", "module"])
)

print("\nModule daily summary (long format):")
print(module_daily_summary.head())

# Also make a wide "report" view: date x module (mean energy)
module_mean_energy_wide = module_daily_summary.pivot_table(
    index="date", columns="module", values="mean_energy", aggfunc="mean"
).sort_index().sort_index(axis=1)

print("\nMean energy per module (wide):")
print(module_mean_energy_wide.head())

# -----------------------------
# 6) Reshaping for plotting: melt
# -----------------------------
counts_long = daily_counts.reset_index().melt(
    id_vars="date",
    var_name="detector",
    value_name="n_events"
)

temps_long = daily_mean_temp.reset_index().melt(
    id_vars="date",
    var_name="probe",
    value_name="mean_temp_C"
)

# -----------------------------
# 7) Quick plots (pandas/matplotlib)
# -----------------------------
# Plot A: Daily counts by detector
ax = daily_counts.plot(kind="line", marker="o", figsize=(10, 4))
ax.set_title("Daily event counts per detector")
ax.set_xlabel("Date")
ax.set_ylabel("Count")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Plot B: Daily mean temperature per probe
ax = daily_mean_temp.plot(kind="line", marker="o", figsize=(10, 4))
ax.set_title("Daily mean temperature per probe")
ax.set_xlabel("Date")
ax.set_ylabel("Temperature (°C)")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Plot C: Mean calibrated energy per module
ax = module_mean_energy_wide.plot(kind="line", marker="o", figsize=(10, 4))
ax.set_title("Daily mean calibrated energy per module")
ax.set_xlabel("Date")
ax.set_ylabel("Mean energy (arb. units)")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# -----------------------------
# 8) Final "Report" objects students would submit
# -----------------------------
report = {
    "daily_mean_temp": daily_mean_temp,
    "daily_counts": daily_counts,
    "module_daily_summary_long": module_daily_summary,
    "module_mean_energy_wide": module_mean_energy_wide,
    "counts_long": counts_long,
    "temps_long": temps_long,
}

print("\nReport keys:", list(report.keys()))
```
`````